# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Run overnight

In [ ]:
# # Predict TAFT vs GUAT
# for monkey_name in ['monkey_Bruno', 'monkey_Schro']:
#     cgtxs = GUAT_vs_TAFT_x_sessions_class.GUATandTAFTacrossSessionsClass()
#     combd_GUAT_x_df, combd_TAFT_x_df = cgtxs.streamline_getting_combd_GUAT_or_TAFT_x_df(monkey_name=monkey_name)

# # Predict num_stops
# combined_info_exists_ok = True
# for monkey_name in ['monkey_Bruno', 'monkey_Schro']:
#     gc_kwargs_temp = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()
#     gc_kwargs_temp['num_old_ff_per_row'] = 2
#     gc_kwargs_temp['num_new_ff_per_row'] = 3

#     gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp, monkey_name=monkey_name)
#     gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=combined_info_exists_ok, 
#                                     traj_df_exist_in_GUAT_store_ok=True)

# predict TAFT vs GUAT

## data across sessions

In [ ]:
monkey_name='monkey_Bruno'
cgtxs = GUAT_vs_TAFT_x_sessions_class.GUATandTAFTacrossSessionsClass()
# now, put the data into a GUATvsTAFTclass instance for downstream analysis
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass()
cgt.GUAT_x_df, cgt.TAFT_x_df = cgtxs.streamline_getting_combd_GUAT_or_TAFT_x_df(monkey_name=monkey_name)

## data for one session

In [ ]:
exists_ok = True
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass(ref_point_mode='time', 
                                            raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330',
                                            ref_point_value=-1.5,)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='TAFT', exists_ok=exists_ok)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='GUAT', exists_ok=exists_ok)


In [ ]:
# # test and see what is taking so long in running a function
# import cProfile

# cProfile.run("cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='GUAT', exists_ok=False)", sort='cumtime')

# #ncalls  tottime  percall  cumtime  percall

## ML

# exp

### get data & vif

In [ ]:
cgt.get_x_and_y_var_df()

eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in cgt.x_var_df.columns if any(k in col for k in eye_keywords)]
cgt.x_var_df = cgt.x_var_df.drop(columns=eye_cols)


from neural_data_analysis.neural_analysis_tools.model_neural_data import design_bases
x_var_df_reduced = design_bases.reduce_df(cgt.x_var_df)

ml_inst = ml_methods_class.MlMethods(x_var_df=x_var_df_reduced,
                                     y_var_df=cgt.y_var_df)

In [ ]:
ml_inst.x_var_df.shape

In [ ]:
# can skip this if only wanting ML results
pd.set_option('display.max_rows', 100)
ml_inst.use_vif(ml_inst.x_var_df)
features_w_big_vif = ml_inst.vif_df[ml_inst.vif_df['vif'] > 100].feature.values
#ml_inst.x_var_df = ml_inst.x_var_df.drop(columns=features_w_big_vif)

In [ ]:
ml_inst.vif_df.head(20)

In [ ]:
# specific_columns = ml_inst.vif_df[ml_inst.vif_df["VIF"] > 2000].feature.values
# ml_inst.show_correlation_heatmap(specific_columns=specific_columns)
ml_inst.show_correlation_heatmap()

In [ ]:
df = ml_inst.x_var_df

In [ ]:
# [col for col in df.columns if 'cluster' not in col]

### run ML

In [ ]:
ml_inst.x_var_df.columns[3]

In [ ]:
ml_inst.use_ml_model_for_classification(ml_inst.x_var_df, ml_inst.y_var_df)


# feature selection

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(ml_inst.x_var_df, ml_inst.y_var_df)
important = model.coef_[0] != 0
X_new = ml_inst.x_var_df.loc[:, important]
X_new.columns


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
sfs = SequentialFeatureSelector(model, n_features_to_select=10, direction="forward")
X_new = sfs.fit_transform(ml_inst.x_var_df, ml_inst.y_var_df.values.ravel())


In [ ]:
X_new

In [ ]:

# Boolean mask of selected features
mask = sfs.get_support()

# Names of selected features
selected_features = ml_inst.x_var_df.columns[mask]

print("Selected features:")
print(selected_features)

## grad_boosting (so that we can see feature importance)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ml_inst.x_var_df, ml_inst.y_var_df, test_size=0.2, random_state=42)

# Define the model
model = GradientBoostingClassifier(
    learning_rate=0.05, max_depth=7, max_features='sqrt',
    min_samples_leaf=2, min_samples_split=7,
    n_estimators=500, subsample=0.5
)


# Fit the model
model.fit(X_train, y_train)

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': ml_inst.x_var_df.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 18))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.title('Feature Importances')
plt.show()

# Determine significant features (e.g., importance > 0.01)
significant_features = feature_importances_df[feature_importances_df['Importance'] > 0.01]
print("Significant features:")
print(significant_features)

stop !

## Compare distributions

In [ ]:
TAFT = cgt.TAFT_x_df.copy()
TAFT['whether_TAFT'] = 1
GUAT = cgt.GUAT_x_df.copy()
GUAT['whether_TAFT'] = 0
both_df = pd.concat([TAFT, GUAT], axis=0)

In [ ]:

# for each feature in significant_features, plot the histogram of the feature for each class
for feature in ['cur_ff_distance_at_ref', 'cur_ff_angle_at_ref']:
    sns.histplot(x=feature, data=both_df, stat='probability', kde=False, hue='whether_TAFT', common_norm=False)
    plt.title(f'{feature} histogram')
    plt.show()

In [ ]:
# for each feature in significant_features, plot the histogram of the feature for each class
max_features_to_plot = 3
count = 0
for feature in significant_features['Feature']:
    sns.histplot(x=feature, data=both_df, stat='probability', kde=False, hue='whether_TAFT', common_norm=False)
    plt.title(f'{feature} histogram')
    plt.show()
    count += 1
    if count >= max_features_to_plot:
        break

### exp

In [ ]:
cgt.TAFT_trials_df

In [ ]:
cgt.GUAT_w_ff_df

### plot trials

In [ ]:
GUAT_or_TAFT = 'GUAT'
list_of_stop_point_index = cgt.GUAT_x_df[cgt.GUAT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
GUAT_or_TAFT = 'TAFT'
list_of_stop_point_index = cgt.TAFT_x_df[cgt.TAFT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
sub = cgt.take_out_subsets_to_plot(list_of_stop_point_index, GUAT_or_TAFT)

In [ ]:
counter = 0
for i, row in sub.iterrows():
    if GUAT_or_TAFT == 'GUAT':
        ff_column = 'latest_visible_ff'
    else:
        ff_column = 'ff_index'

    print('cur_ff_distance_at_ref', row['cur_ff_distance_at_ref'])
    additional_kwargs = {'point_indices_to_be_marked': row['stop_point_index'],
                         'point_indices_to_be_marked_3rd_kind': row['ref_point_index']
                         }
    
    plotting_kwargs_temp = copy.deepcopy(cgt.plotting_kwargs)
    plotting_kwargs_temp['indices_of_ff_to_mark'] = [row[ff_column]]
    trial = row['trial']
    duration = [cgt.ff_caught_T_new[trial-2], cgt.ff_caught_T_new[trial]+1]
    returned_info = plot_trials.PlotTrials(duration, 
                *cgt.PlotTrials_args,
                **plotting_kwargs_temp,   
                **additional_kwargs          
                )
    plt.show()
    # process_GUAT_trials_class.ProcessGUATtrials.make_GUAT_plot(cgt, trial, [row[ff_column]], np.arange(row['stop_point_index']-200, row['stop_point_index']+300),
    #                                                            additional_kwargs=additional_kwargs)
    if counter == 3:
        break
    counter += 1

# More plots

## get data

In [ ]:
pretty_null_arc_plot_kwargs = {'player': 'monkey',
        'show_believed_target_positions': True,                       
        'show_stops': True,
        'show_reward_boundary': True,
        'show_scale_bar': True,
        'hitting_arena_edge_ok': True,
        'trial_too_short_ok': True,
        'show_alive_fireflies': False,
        'adjust_xy_limits': True,
        'zoom_in': True,
        'minimal_margin': 60,
        }


In [ ]:
cgt.get_relevant_monkey_data(already_retrieved_ok=True)
cgt.find_patterns()
cgt.make_PlotTrials_args()
cgt.make_or_retrieve_target_clust_last_vis_df()

cgt.make_curvature_df()
cgt.find_opt_arc_info_from_curvature_df()
cgt.make_best_arc_df()
cgt.get_elements_for_plotting()

In [ ]:
cgt.best_arc_df.columns.tolist()

In [ ]:
reload(show_null_trajectory)
cgt.null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(cgt.best_arc_df, cgt.monkey_information, column_for_color=None, ignore_error=True)
#cgt.null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(cgt.best_arc_df, cgt.monkey_information, column_for_color='diff_percentile_in_decimal')

# also get cgt.null_arc_to_center_info_for_plotting
all_point_index = cgt.best_arc_df.point_index.values
all_ff_index = cgt.best_arc_df.ff_index.values
remaining_index_of_array, remaining_all_ff_index, remaining_all_point_index = show_null_trajectory.eliminate_invalid_ff_for_null_arc(all_ff_index, all_point_index, cgt.ff_real_position_sorted, cgt.monkey_information)
cgt.find_and_package_arc_to_center_info_for_plotting(remaining_all_point_index, remaining_all_ff_index)

# Try a few times vs. give up after trying

## Compared with the rest

In [ ]:
reload(decision_making_class)
reload(plot_trials)
reload(intended_targets_classes)

try_a_few_times_trials = cgt.try_a_few_times_trials
target_cluster_TAFT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
sort_1_df = target_cluster_TAFT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Try a few times"


target_cluster_no_flash_after_stopn_TAFT = cgt.target_clust_last_vis_df[~cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
sort_2_df = target_cluster_no_flash_after_stopn_TAFT
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Non-TAFT"

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,                                                     
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:
reload(show_null_trajectory)
reload(decision_making_class)
reload(plot_trials)
reload(intended_targets_classes)
reload(category_class)


category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
category_item.make_histogram_of_distances_from_previous_targets()
category_item.make_polar_plot_of_positions_from_previous_targets()


category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
category_item.plot_distributions_of_visible_ff_and_in_memory_ff()


## visualize predictions

In [ ]:
gnb = GaussianNB()
# gnb.fit(ml_inst.x_var_df, ml_inst.y_var_df)
#(trained_model = gnb, max_plot_to_make = 2)

#category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## plot_trajectories

In [ ]:
current_i = 30

In [ ]:
reload(specific_utils)
reload(plot_trials)
category_item.plot_trajectories(trials=category_item.sort_1_trials[current_i:current_i+2], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
current_i = current_i+2

## 724

In [ ]:
cgt.null_arc_to_center_info_for_plotting[cgt.null_arc_to_center_info_for_plotting['arc_point_index']==112997]

In [ ]:
pretty_null_arc_plot_kwargs['show_ff_indices'] = False

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1856.504211, 1862.693883]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([908, 723, 724])
intended_target_ff_index = None #749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


# For ppt

## TAFT (retry)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1856.504211, 1862.693883]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([908, 723, 724])
intended_target_ff_index = None #749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

pretty_null_arc_plot_kwargs_temp.update({'show_start': False})

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3

updated_args = {'title': ''}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        **updated_args)


## GUAT (switch)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1537.023148, 1542.046287]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([598, 599, 627])
intended_target_ff_index = 717
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

pretty_null_arc_plot_kwargs_temp.update({'show_start': False})

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3

updated_args = {'title': ''}

show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= False, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        **updated_args)



# Give up after trying

## GUAT_cluster_df

In [ ]:
## if I want to use manual_anno for GUAT_item
# dm = decision_making_class.DecisionMaking(raw_data_folder_path=raw_data_folder_path)
# dm.retrieve_manual_anno()
# manual_anno = dm.manual_anno

In [ ]:
reload(process_GUAT_trials_class)
auto_annot, auto_annot_long = decision_making_utils.make_auto_annot(cgt.best_arc_df, cgt.monkey_information, cgt.ff_caught_T_new)

GUAT_item = process_GUAT_trials_class.ProcessGUATtrials(cgt.give_up_after_trying_info_bundle, \
                                                        cgt.PlotTrials_args, max_distance_to_stop_for_GUAT_target=50, max_allowed_time_since_last_vis=2.5)
GUAT_item.find_possible_objects_of_pursuit_in_GUAT()
GUAT_item.find_GUAT_ff_aimed_at_from_manual_anno(auto_annot)
#GUAT_item.check_GUAT_object_with_manual_anno(verbose=False)
#GUAT_item.inspect_clusters_w_o_matching_ff()
GUAT_item.make_GUAT_cluster_df()

## Compare with TAFT

In [ ]:
try_a_few_times_trials = cgt.try_a_few_times_trials
give_up_after_trying_trials = cgt.give_up_after_trying_trials

target_cluster_TAFT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]
target_cluster_GUAT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]

sort_1_df = target_cluster_TAFT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Try a few times"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Give up after trying"
sort_2_ff_indices = GUAT_item.GUAT_cluster_df.last_vis_ff_index.values

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,
                                                              sort_2_ff_indices = sort_2_ff_indices,
                                                              sort_2_ff_positions = cgt.ff_real_position_sorted[sort_2_ff_indices],                                                             
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:

category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
category_item.make_histogram_of_distances_from_previous_targets()
category_item.make_polar_plot_of_positions_from_previous_targets()

category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
category_item.plot_distributions_of_visible_ff_and_in_memory_ff()
## the below function (free selection prediction) shouldn't be used on GUAT. So far, it should only be used on VBLO.
# category_item.make_and_visualize_free_selection_predictions_using_trained_model(trained_model = gnb, max_plot_to_make = 99)
# category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## Compared with the rest

In [ ]:
give_up_after_trying_trials = cgt.give_up_after_trying_trials

target_cluster_no_flash_after_stopn_GUAT = cgt.target_clust_last_vis_df[~cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]
target_cluster_GUAT = cgt.target_clust_last_vis_df[cgt.target_clust_last_vis_df['target_index'].isin(give_up_after_trying_trials)]

sort_1_df = target_cluster_no_flash_after_stopn_GUAT
sort_1_trials = sort_1_df.target_index.values
sort_1_name = "Non-GUAT"

sort_2_df = GUAT_item.GUAT_cluster_df
sort_2_trials = sort_2_df.target_index.values
sort_2_name = "Give up after trying"
sort_2_ff_indices = GUAT_item.GUAT_cluster_df.last_vis_ff_index.values
sort_2_time_for_predicting_ff = GUAT_item.GUAT_cluster_df.latest_visible_time_before_last_stop.values

category_item = category_class.ProcessCategoryData(sort_1_trials=sort_1_trials, sort_2_trials=sort_2_trials,
                                                              sort_1_name=sort_1_name, sort_2_name=sort_2_name, 
                                                              sort_1_df=sort_1_df, sort_2_df=sort_2_df,
                                                              sort_2_ff_indices = sort_2_ff_indices,
                                                              sort_2_ff_positions = cgt.ff_real_position_sorted[sort_2_ff_indices],   
                                                              sort_2_time_for_predicting_ff = sort_2_time_for_predicting_ff,                                                       
                                                              PlotTrials_args=cgt.PlotTrials_args, ff_flash_sorted=cgt.ff_flash_sorted,
                                                              null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)

In [ ]:
category_item.clean_out_cross_boundary_trials()
category_item.clean_out_trials_where_target_cluster_was_not_seen_for_a_long_time_before_capture()
category_item.make_polar_plot_of_target_last_seen_positions()
category_item.make_histograms_of_target_last_seen_attributes()
category_item.make_histogram_of_distances_from_previous_targets()
category_item.make_polar_plot_of_positions_from_previous_targets()
category_item.plot_trajectories(trials=category_item.sort_1_trials[17:18], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting)
category_item.plot_distributions_of_visible_ff_and_in_memory_ff()
## The function below (free selection prediction) shouldn't be used on GUAT. So far, it should only be used on VBLO.
#category_item.make_and_visualize_free_selection_predictions_using_trained_model(trained_model = gnb, use_sort_1=False, use_sort_2=True, max_plot_to_make=3)
# category_item.inspect_special_cases(weird_trials=[98, 180, 212, 649])

## plot_trajectories

In [ ]:
current_i = 60

In [ ]:
reload(specific_utils)
reload(plot_trials)
cgt.null_arc_info_for_plotting_temp = cgt.null_arc_info_for_plotting.copy()
cgt.null_arc_info_for_plotting_temp['show_connect_path_ff'] = True
category_item.plot_trajectories(trials=category_item.sort_2_trials[current_i:current_i+2], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting_temp)
current_i = current_i+2

## 489 (Bruno 0330)

In [ ]:

reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1254, 1262]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1042, 1241, 489, 685, 744])
intended_target_ff_index = 1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, show_ff_indices=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 567 ??

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1469.670162, 1475.0996949999999]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([780, 703, 655, 567])
intended_target_ff_index = None #1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 575 ?

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1483.929318, 1489.226027]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([575, 576, 624, 1620, 1170, 1351])
intended_target_ff_index = None #1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, show_ff_indices=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 586

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1504.795097, 1512.714639]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([595])
intended_target_ff_index = 595
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 599 ??

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1537.023148, 1542.046287]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([598, 599, 627])
intended_target_ff_index = 717
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 3
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )



## 622

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1594.507002, 1597.611809]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1351, 831, 856])
intended_target_ff_index = 856
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = duration_to_plot[0] + 1
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 636

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1625.433097, 1629.310465]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1464, 878, 636, 974])
intended_target_ff_index = 636
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 641

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1642, 1647]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([726, 665])
intended_target_ff_index = 726
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 641 (2) x

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1646, 1652]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([822, 641, 665])
intended_target_ff_index = 822
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 650 ?

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1666.310034, 1671.855963]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1089, 694, 650])
intended_target_ff_index = None#822
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 653

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1674.543492, 1681.62897]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([708, 660, 820])
intended_target_ff_index = 820
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 653 (2)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1677.543492, 1683.62897]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([708, 690, 771, 653, 789])
intended_target_ff_index = 708
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True

pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## 658 (too hard to see)

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1684.973395, 1692.555617]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([668, 656, 657, 704, 797, 798, 824, 658])
intended_target_ff_index = None # 658
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 660 

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1692.487749, 1697.950517]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([751, 660, 820, 661])
intended_target_ff_index = 751
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=400, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 679

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1752.16371, 1761.527301]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array()
intended_target_ff_index = 728
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=450, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 693

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1791.986039, 1798.079852]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([692, 693, 724])
intended_target_ff_index = 724
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=450, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 713

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1831, 1836.149079]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([692, 693, 724])
intended_target_ff_index = 1492
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=330, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 718

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1838, 1843.994575]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([717, 1401, 718, 845, 905])
intended_target_ff_index = 1401
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 723

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1849.504211, 1854.4]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([722, 749])
intended_target_ff_index = 749
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 730

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1873.116986, 1878.281488]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([1106])
intended_target_ff_index = 1106
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 751

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1919.422092, 1926.644391]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([876, 751])
intended_target_ff_index = 876
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 756

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1934.510997, 1943.393497]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = np.array([756, 1089, 1703])
intended_target_ff_index = 1089
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


## 765

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
# update the figure size of rcParams
plt.rcParams['figure.figsize'] = [9, 9]
duration_to_plot = [1959.975111, 1968.49221]

max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([756, 1089, 1703])
intended_target_ff_index = None #1089
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = True
#pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_specific_indices'] = np.array([668, 656, 657, 704, 797, 798, 824, 658])
pretty_null_arc_plot_kwargs_temp['vary_color_for_connecting_path_ff'] = False
pretty_null_arc_plot_kwargs_temp['show_reward_boundary'] = False
pretty_null_arc_plot_kwargs_temp['how_to_show_ff_for_visible_segments'] = 'circle'

time_to_begin_plotting_null_arc = None
time_between_every_two_null_arcs = 0.3


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=230, ff_max_distance_to_path=400, time_between_every_two_null_arcs=time_between_every_two_null_arcs, time_to_begin_plotting_null_arc=time_to_begin_plotting_null_arc, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )


In [ ]:
category_item.plot_trajectories(trials=[765], null_arc_info_for_plotting=cgt.null_arc_info_for_plotting,
                                show_connect_path_ff=True, show_ff_indices=True)

In [ ]:
reload(show_null_trajectory)
duration_to_plot = [1254.09006, 1260.797728]
max_num_plot_to_make = 1
ff_indices_to_plot_null_arc = None #np.array([62, 413, 60, 61])
intended_target_ff_index = 1241
point_indices_to_plot_null_arc = None #[10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, cgt.best_arc_df, cgt.monkey_information, cgt.ff_dataframe, cgt.null_arc_info_for_plotting, cgt.null_arc_to_center_info_for_plotting, cgt.PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs= True, ff_max_distance_to_intended_target=250, ff_max_distance_to_path=400, time_between_every_two_null_arcs=0.2, time_to_begin_plotting_null_arc=None, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, point_indices_to_plot_null_arc=point_indices_to_plot_null_arc, ff_indices_to_plot_null_arc=ff_indices_to_plot_null_arc, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

In [ ]:
STOP!

# Appendix

## Plot trials

In [ ]:
classic_plot_kwargs = {'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True}
cgt.PlotTrials_args = (cgt.monkey_information, cgt.ff_dataframe, cgt.ff_life_sorted, cgt.ff_real_position_sorted, cgt.ff_believed_position_sorted, cgt.cluster_around_target_indices, cgt.ff_caught_T_new)

In [ ]:
from data_wrangling import general_utils

In [ ]:

num_trials = 2 


with general_utils.initiate_plot(10,10,100):
  for currentTrial in [96+1, 102+1]: #two_in_a_row[10:20]:
      duration = [cgt.ff_caught_T_new[currentTrial-num_trials], cgt.ff_caught_T_new[currentTrial]]
      plot_trials.PlotTrials(duration, 
                  *cgt.PlotTrials_args,
                  **classic_plot_kwargs,
                  currentTrial = currentTrial,
                  num_trials = num_trials,                   
                  show_connect_path_ff_except_targets = True,
                  show_path_when_target_visible = True,
                  show_ff_indices = False,
                  hitting_arena_edge_ok = True,
                  visible_distance = 400,
                  minimal_margin = 300,
                  show_monkey_angles = True
                  )
      plt.show()  

      